<a href="https://colab.research.google.com/github/helsonmatos/EstudosRN/blob/main/RedeNeuralGanKeras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.models import Sequential, Model
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam
import sys

import numpy as np
import matplotlib.pyplot as plt

(x_treino,_),(_,_) = mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [2]:
x_treino.shape

(60000, 28, 28)

In [3]:
x_treino = x_treino / 127.5 - 1.0
x_treino = np.expand_dims(x_treino, axis=3)

In [4]:
x_treino.shape

(60000, 28, 28, 1)

In [5]:
class DCGAN():
    def __init__(self):
        # input shape
        self.altura = 28
        self.largura = 28
        self.channels = 1
        self.img_shape = (self.altura, self.largura, self.channels)
        self.dimensao_randomica = 100

        optimizer = Adam(learning_rate=0.0002, beta_1=0.5)

        #Definindo o discriminator:
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

        #Definindo generator:
        self.generator = self.build_generator()

        #Definindo o modelo completo:
        esqueleto_entrada_randomica = Input(shape=(self.dimensao_randomica,))
        esqueleto_imagem_gerada = self.generator(esqueleto_entrada_randomica)

        self.discriminator.trainable = False # impedindo o discriminator de treinar
        verificacoes = self.discriminator(esqueleto_imagem_gerada)

        self.gan_completa = Model(esqueleto_entrada_randomica, verificacoes)
        self.gan_completa.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_generator(self):

        modelo = Sequential()

        # O objeto é finalizar com uma dimensão 28 x 28 x 1:
        modelo.add(Dense(128 * 7 * 7, activation="relu", input_dim=self.dimensao_randomica))
        modelo.add(Reshape((7, 7, 128)))
        modelo.add(UpSampling2D())
        modelo.add(Conv2D(128, kernel_size=3, padding="same"))
        modelo.add(BatchNormalization(momentum=0.8))
        modelo.add(Activation("relu"))
        modelo.add(UpSampling2D())
        modelo.add(Conv2D(64, kernel_size=3, padding="same"))
        modelo.add(BatchNormalization(momentum=0.8))
        modelo.add(Activation("relu"))
        modelo.add(Conv2D(self.channels, kernel_size=3, padding="same"))
        modelo.add(Activation("tanh"))

        ruido = Input(shape=(self.dimensao_randomica,))
        imagem_gerada = modelo(ruido)

        return Model(ruido, imagem_gerada)

    def build_discriminator(self):

        modelo = Sequential()

        modelo.add(Conv2D(32, kernel_size=3,strides=2, input_shape=self.img_shape, padding="same"))
        modelo.add(LeakyReLU(alpha=0.2))
        modelo.add(Dropout(0.25))
        modelo.add(Conv2D(64, kernel_size=3,strides=2, padding="same"))
        modelo.add(ZeroPadding2D(padding=((0,1),(0,1))))
        modelo.add(BatchNormalization(momentum=0.8))
        modelo.add(LeakyReLU(alpha=0.2))
        modelo.add(Dropout(0.25))
        modelo.add(Conv2D(128, kernel_size=3,strides=2, padding="same"))
        modelo.add(BatchNormalization(momentum=0.8))
        modelo.add(LeakyReLU(alpha=0.2))
        modelo.add(Dropout(0.25))
        modelo.add(Conv2D(256, kernel_size=3,strides=1, padding="same"))
        modelo.add(BatchNormalization(momentum=0.8))
        modelo.add(LeakyReLU(alpha=0.2))
        modelo.add(Dropout(0.25))
        modelo.add(Flatten())
        modelo.add(Dense(1, activation='sigmoid'))

        imagem_entrada = Input(shape=self.img_shape)
        verificacao = modelo(imagem_entrada)

        return Model(imagem_entrada, verificacao)
    
    def train(self, epochs, batch_size=128, intervalo_registro=50, x_treino=None):

        x_treino = x_treino # carregando o dataset

        # Criando os gabaritos (img reais = 1, imgs fake = 0)
        uns = np.ones((batch_size,1))
        zeros = np.zeros((batch_size,1))

        for epoch in range(epochs):
            "Discriminator"
            # Selecionando um batch aleatório de imagens reais:
            indices = np.random.randint(0, x_treino.shape[0],batch_size)
            imagens_reais = x_treino[indices]

            # Selecionando um batch de ruídos para gerar imagens fake:
            amostras_ruidos = np.random.normal(0,1, (batch_size, self.dimensao_randomica))
            imagens_geradas = self.generator.predict(amostras_ruidos)

            # Treinando o discriminator para aprender a diferença entre imagens reais e fake:
            d_loss_real = self.discriminator.train_on_batch(imagens_reais, uns)
            d_loss_fake = self.discriminator.train_on_batch(imagens_geradas, zeros)
            discriminator_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            "Generator"
            # Treinando o generator para aprender a enganar o discriminator:
            generator_loss = self.gan_completa.train_on_batch(amostras_ruidos, uns)

            # Mostrando o progresso:
            print(f"{epoch} [Discrimator loss: {discriminator_loss[0]}, Acurácia: {100*discriminator_loss[1]:.2f}] [Generator loss: {generator_loss}]")

            # Salvando as imagens geradas:
            if epoch % intervalo_registro == 0:
                self.save_imgs(epoch)
    def save_imgs(self, epoch):
        linhas = 5
        colunas = 5
        amostras_ruidos = np.random.normal(0, 1, (linhas * colunas, self.dimensao_randomica))
        imagens_geradas = self.generator.predict(amostras_ruidos)

        # Normalizando as imagens entre [0,1] para mostrar com o matplotlib gray:
        imagens_geradas = 0.5 * imagens_geradas + 0.5

        fig, axs = plt.subplots(linhas, colunas)
        cont = 0
        for i in range(linhas):
            for j in range(colunas):
                axs[i,j].imshow(imagens_geradas[cont,:,:,0], cmap='gray')
                axs[i,j].axis('off')
                cont +=1
        fig.savefig("/content/drive/MyDrive/Colab Notebooks/mnist_%d.png" % epoch)
        plt.close()

In [ ]:
import time
gan_convolucional = DCGAN()
inicio = time.time()
gan_convolucional.train(epochs=20000, batch_size=32, intervalo_registro=500, x_treino=x_treino)
print(f"Tempo em minutos: {(time.time() - inicio)/60}")